<a href="https://colab.research.google.com/github/strange-hawk/tensorflow_examples/blob/main/boston_house_regression_tf_estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow.feature_column as fc
from tensorflow.keras.datasets import boston_housing

In [22]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [23]:
features= ['CRIM' ,'ZN', 'INDUS', 'CHAS','NOX','RM','AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

In [24]:
x_train_df = pd.DataFrame(x_train, columns=features)
y_train_df = pd.DataFrame(y_train, columns=['MEDV'])
x_test_df = pd.DataFrame(x_test, columns=features)
y_test_df = pd.DataFrame(y_test, columns=['MEDV'])

In [ ]:
x_train_df

In [10]:
import seaborn as sns

In [ ]:
sns.pairplot(x_train_df.corr()[:5])

In [ ]:
sns.pairplot(x_train_df.corr().iloc[:5, :5])

In [28]:
np.sum(x_train_df.isna())

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
dtype: int64

In [30]:
feature_column = []
for i in features:
  feature_column.append(fc.numeric_column(i,dtype=tf.float32))

In [31]:
feature_column

[NumericColumn(key='CRIM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='ZN', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='INDUS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='CHAS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='NOX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='RM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='AGE', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='DIS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='RAD', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='TAX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PTRATIO'

In [52]:
def estimator_input_func(df_data, df_label, epochs = 10, shuffle = True, batch_size = 32):
  def input_func():
    dataset = tf.data.Dataset.from_tensor_slices((dict(df_data),df_label))
    if shuffle:
      dataset = dataset.shuffle(100)
    dataset = dataset.batch(batch_size).repeat(epochs)
    return dataset
  return input_func


In [53]:
train_input_func = estimator_input_func(x_train_df, y_train_df)
val_input_func = estimator_input_func(x_test_df, y_test_df, epochs=1, shuffle=False)

In [68]:
lin_reg = tf.estimator.LinearRegressor(feature_columns=feature_column, model_dir='logs/func/')
lin_reg.train(train_input_func, steps=100)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'logs/func/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in 

In [69]:
t = lin_reg.predict(val_input_func)

In [ ]:
for pred,exp in zip(t, y_test[:32]):
  print(pred['predictions'][0],exp)

In [70]:
for i,j in zip(t,y_test[:32]):
  print(i['predictions'][0] , j)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from logs/func/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
7.1122203 7.2
25.807232 18.8
23.10134 19.0
25.391039 27.0
23.884884 22.2
23.775253 24.5
31.18158 31.2
26.886604 22.9
21.525688 20.5
27.488205 23.2
11.584031 18.6
24.648788 14.5
25.506187 17.8
24.011974 50.0
22.030474 20.8
26.366308 24.3
22.141209 2